# Natural Language Querying (NLQ)

Using the `.with_natural_query` clause in queries presents the opportunity to describe your query using natural language. The underlying solution (currently using models from OpenAI, local LLM support coming soon) fills the query parameters (weights, filters and other query-time inputs) based on the user's natural language query and runs it.

In [1]:
%pip install superlinked==12.18.1

## Create your configuration

In [2]:
import os
import pandas as pd
from superlinked import framework as sl


class Product(sl.Schema):
    id: sl.IdField
    description: sl.String
    stars: sl.Float
    type: sl.String


product = Product()

description_space = sl.TextSimilaritySpace(
    text=product.description, model="Alibaba-NLP/gte-large-en-v1.5"
)
rating_maximizer_space = sl.NumberSpace(
    number=product.stars, min_value=1.0, max_value=5.0, mode=sl.Mode.MAXIMUM
)
index = sl.Index([description_space, rating_maximizer_space], fields=[product.type])

## Define a natural language powered query

In [3]:
# fill this with your API key - this will drive param extraction
openai_config = sl.OpenAIClientConfig(
    api_key=os.environ["OPEN_AI_API_KEY"], model="gpt-4o"
)

# it is possible now to add descriptions to a `Param` to aid the parsing of information from natural language queries.
text_similar_param = sl.Param(
    "query_text",
    description=(
        "The text in the user's query that is used to search in the products' description."
        " Extract info that does not apply to other spaces or params."
    ),
)

# Define your query using dynamic parameters for query text and weights.
# let's create a base query that we will modify to 2 alternative versions
base_query = (
    sl.Query(
        index,
        weights={
            description_space: sl.Param("description_weight"),
            rating_maximizer_space: sl.Param("rating_maximizer_weight"),
        },
    )
    .find(product)
    .similar(
        description_space,
        text_similar_param,
        sl.Param("description_similar_clause_weight"),
    )
    .limit(sl.Param("limit"))
    # we will have our LLM fill them based on our natural language query
    .with_natural_query(sl.Param("natural_query"), openai_config)
)


query = (
    base_query
    # it is possible - but not necessary - to add descriptions to Params, to help the extraction
    .filter(
        product.type
        == sl.Param(
            "filter_by_type",
            description="Used to only present items that have a specific type. Possible values are product, or book.",
        )
    )
)

query_with_default_filter = base_query.filter(
    product.type
    == sl.Param(
        "filter_by_type",
        description="Used to only present items that have a specific type. Possible values are product, or book.",
        default="product",  # we can add defaults so if the NLQ is not able to fill a Param, it will use this default
    )
)

## Load test data

In [4]:
# Run the app.
source: sl.InMemorySource = sl.InMemorySource(
    product,
    parser=sl.DataFrameParser(schema=product, mapping={product.id: "product_id"}),
)
executor = sl.InMemoryExecutor(sources=[source], indices=[index])
app = executor.run()

# Download dataset.
df = pd.read_csv(
    "https://storage.googleapis.com/superlinked-sample-datasets/esci_sample.csv",
)

In [5]:
# Ingest data to the framework.
source.put([df])

## Run queries using natural language

Filling up params using descriptions of natural language. The only exceptions are:
- limit
- radius
- the input to the natural query itself.

### Params without default value set by the user

In [6]:
pd.set_option("display.max_colwidth", 500)

In [7]:
positive_results = app.query(
    query,
    natural_query="good design products",
    limit=10,
)
positive_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,Article №: Keith Ti5820 ● FUNCTIONALLY SIMPLE AND ELEGANT --- Much of our design inspiration comes directly from outdoor adventurers and travelers. We seek out innovative solutions for their needs with the belief that less is more and that simplicity yields elegance. ● STRAIGHTFORWARDLY VERSATILE AND MULTI-PURPOSE --- We design our products to be used in ways expected and unexpected to save weight and space when every ounce counts. ● ENVIRONMENTALLY FRIENDLY AND HEALTHY --- We strive to mini...,4.8,product,B01NAEP534,0.676896,0
1,"Inituitive Breath-o-prene ErgoFrom Headgear, Simplus Headgear Med/Lrg",4.7,product,B01DJGEERQ,0.647900,1
2,Adidas logo above back right pocket\nUPF 50+ for Sun protection,4.7,product,B078W8M374,0.643395,2
3,"We specialize in wall and tabletop frames, document frames, shadow boxes, organizer box, and other home and office decor. We pride ourselves on creating classic, and meaningful products that will make your home or office shine as well as providing items that are ready to give as giffts straight out of the box. Time Does Not Wait for You! Yesterday will not be called again. We always inadvertently remembered the good moments of the past. Let your photographs and pictures seize the day, for fl...",4.5,product,B089SQX1R1,0.641352,3
4,"SPECIFICATIONS -Material : Stainless Steel-Thickness : 4mm -Finish : Matte Black -Installation Method : Screw fixing Package Includes 2 x Brackets 8 X Screws Buy from Alise Solid metal construction, 4mm-thick, heavy duty design Solid stainless steel materials, Choose 4mm-Thick stainless steel brackets,Durable and rust-free Exclusive Brand, Production and Marketing Integration,Quality Assurance,We choose the best materials, the most exquisite product technology, and the best quality s...",4.6,product,B07L879PDS,0.638578,4
5,"Low-profile design offers comfort and security\nHigh-tech memory foam comfortably conforms to your neck and shoulder, antimicrobial material allows moisture to dissipate so you stay cool\nSlimlock secures to your guitar or bass without the bulk and rigidity of a conventional connector\nFully adjustable 37""-48"" (94cm-122cm)\nMade in the USA, Date First Available: April 2, 2007, Customer Reviews: 5.0 out of 5 stars 2 ratings 5.0 out of 5 stars, Size: Regular, Item model number: 8202162, ASIN: ...",5.0,product,B000G1KDJ6,0.635408,5
6,"These Can Coolers Are White Blank Sublimation 12 Oz 3MM Insulated Neoprene Material, They Are Foldable And Waterproof., Insulated Neoprene, Date First Available: April 8, 2021, ASIN: B091V8YM2S, Package Dimensions: 7.6 x 7.24 x 1.65 inches, Customer Reviews: 4.6 out of 5 stars 250 ratings 4.6 out of 5 stars, Item Weight: 8.4 ounces",4.6,product,B091V8YM2S,0.634836,6
7,"SKILLED-RIDER SCOOTER: The next generation of the award-winning YBIKE GLX scooter. With its modern 3 wheel Lean-and-Steer design, the GLX PRO is the best option for older and more skilled riders.\nADJUSTABLE HANDLEBAR: As riders develop better coordination, endurance, and overall strength, the GLX PRO grows with them. The adjustable handlebar extends from 28 to 32.5 inches above the deck.\nFAST & SMOOTH: Features larger 125mm front wheels, wide fiber-reinforced non-slip ABS deck, and composi...",4.8,product,B00BTTFK5Q,0.634516,7
8,Adidas logo above back right pocket\nUPF 50+ for Sun protection\nStretch waistband for mobility and comfort,4.6,product,B079HGN1YT,0.634200,8
9,"About Funkrin: Our brand focuses on the thermal insulation series, providing consumers with full-scene, high-quality and differentiated products that are built to last. The thermal insulation series includes lunch boxes, food jar, coffee mug, sports water bottle and other products, to covers consumers' daily thermal insulation needs.Why Choose Us:1.WIDE USE:18oz capacity is ideal for outdoor, travel, party, coffee, water, wine, beverages, milk, etc2. Each tumbler includes a sliding lid, w...",4.3,product,B093DVT1FL,0.631192,9


The parameters extracted by the NLQ solution can be accessed using the `knn_params` attribute of the `Result` object the query returns. Note, that these values are not necessarily absolutely stable if run multiple times as a result of being extracted by an LLM.

In [8]:
positive_results.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'filter_by_type': 'product',
 'query_text': 'good design products',
 'description_similar_clause_weight': 1.0,
 'limit': 10,
 'natural_query': 'good design products'}

If the param is not set explicitly, it will get extracted from the query if possible.

In [9]:
book_filtered_query_result = app.query(
    query, natural_query="acclaimed books about adventures", limit=10
)
book_filtered_query_result.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"*, , Traducción en español del cuento clásico ganador de la Medalla Caldecott, Un día de nieve . Es un día de nieve y Pedro sale a explorar. Mientras pasea entre montones de nieve apilada y observa a los niños grandes jugar a tirarse bolas gigantes--descubre los surcos que dejan sus pies en la nieve espesa y lo divertido que es hacer figuras de ángeles en la nieve. ""This translation of Keats's original Caldecott Medal winner captures the physical and emotional experiences of Peter and his de...",4.9,book,0140543635,0.617732,0
1,"The petite shell-shaped cakes known as madeleines are versatile, pretty, and absolutely delicious. Made famous by Marcel Proust in his novel In Search of Lost Time , this classic French treat is now loved the world over. Beautifully illustrated and lovingly researched, Madeleines features recipes for an incredible variety of flavors and combinations, including such decadent desserts as Dark Chocolate Espresso Madeleines, savory appetizers like Pesto and Pine Nut Madeleines, and showstoppers ...",4.7,book,1594747407,0.612173,1
2,"Embark on a desert adventure with illustrator Geninne D Zlatkis. Crafted as a love letter to the desert, Zlatkis's collection of charming illustrations feature some of her favorite crafty creatures and natural treasures all found in her desert home. Her passion for this magnificent and wild place will inspire you to explore your own color palette and bring the illustrations to vivid life. By the creator of the best-selling Feathered Friends wall calendar. 40 original full-page images illustr...",4.8,book,1631363174,0.612045,2
3,"Mass Market Paperback – January 1, 1995",5.0,book,B0011EU0OA,0.608694,3
4,"NATIONAL BESTSELLER Alexandra Morton has been called ""the Jane Goodall of Canada"" because of her passionate thirty-year fight to save British Columbia's wild salmon. Her account of that fight is both inspiring in its own right and a roadmap of resistance. Alexandra Morton came north from California in the early 1980s, following her first love--the northern resident orca. In remote Echo Bay, in the Broughton Archipelago, she found the perfect place to settle into all she had ever dreamed of: ...",4.8,book,0735279667,0.608444,4
5,"“An absorbing, generous, ravishing book by a high priestess of you-have-to-read-this prose."" ―Cheryl Strayed, author of Wild Pam Houston, an ""early master of the art of rendering fiercely independent, brilliant women in love with the wrong men"" (Sarah Norris, Barnes & Noble Review ), delivers a novel that whisks us from one breathtaking precipice to the next. Along the way, we unravel the story of Pam (a character not unlike the author), a fearless traveler aiming to leave her metaphorical b...",4.0,book,0393343480,0.607346,5
6,"From the writer of the award-winning food blog Harvest and Honey, including Saveur Best Blog finalist Showcasing the flavors and modern cooking techniques of Appalachia and the Blue Ridge Mountains: With over 70 delectable recipes and 80 stunning photographs organized by seasons, Smoke, Roots, Mountain, Harvest is an evocative cookbook rooted in Appalachian ingredients and flavors that takes readers and cooks deep into the heart and soul of America. Lauren McDuffie uses modern cooking techni...",4.5,book,1452168768,0.601266,6
7,"The original, best-selling ""Rock"" book keeps on rolling, now celebrating its 5th anniversary! This modern-day classic follows 14 lovable rocks (and one wise little ladybug) on a heartwarming and hilarious journey, universally loved by children, parents and teachers alike. From zippy little pebbles to big strong boulders, the rock friends get together to play and roll around their favorite hill, only to find that one of their friends, Ricky, can't roll with them. Unlike all of the other rocks...",4.8,book,0578198037,0.600106,7
8,"The world’s most unique guide to alcohol-free drinks. Over 250 pages, featuring nearly 100 delicious, flavorful recipes.

In [10]:
book_filtered_query_result.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'filter_by_type': 'book',
 'query_text': 'adventures',
 'description_similar_clause_weight': 1.0,
 'limit': 10,
 'natural_query': 'acclaimed books about adventures'}

If we set `rating` (or any `Param`) explicitly, it will not be modified based on the natural language query. The solution will extract all information from the query it can, but it will reconcile it with the explicitly set values.

In [11]:
book_explicitly_filtered_results = app.query(
    query,
    natural_query=("Products with high durability. Filter for products."),
    # notice how we explicitly override what is in the natural_query, and that is reflected in the results
    filter_by_type="book",
    limit=10,
)

book_explicitly_filtered_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"110 pages perfect binding 60# offset paper practical size DIN A5 durable softcover Read more, , Paperback – March 8, 2019",4.5,book,1799146049,0.641471,0
1,"One of the most influential works of this century, The Myth of Sisyphus and Other Essays is a crucial exposition of existentialist thought. Influenced by works such as Don Juan and the novels of Kafka, these essays begin with a meditation on suicide, the question of living or not living in a universe devoid of order or meaning. With lyric eloquence, Albert Camus brilliantly posits a way out of despair, reaffirming the value of personal existence, and the possibility of life lived with dignit...",4.7,book,B07HLQQZ3L,0.621295,1
2,"The original, best-selling ""Rock"" book keeps on rolling, now celebrating its 5th anniversary! This modern-day classic follows 14 lovable rocks (and one wise little ladybug) on a heartwarming and hilarious journey, universally loved by children, parents and teachers alike. From zippy little pebbles to big strong boulders, the rock friends get together to play and roll around their favorite hill, only to find that one of their friends, Ricky, can't roll with them. Unlike all of the other rocks...",4.8,book,0578198037,0.618785,2
3,"Newly Revised & Updated! The Eleventh Edition of America’s Best-Selling Dictionary defines the current, active vocabulary of American English and is updated on an ongoing basis. Features more than 225,000 definitions and over 42,000 usage examples. Includes newly added words and meanings across a variety of fields including technology, entertainment, health, science, and society. Special sections include A Handbook of Style, Foreign Words and Phrases, Biographical Names, and Geographical Nam...",4.7,book,0877798079,0.615373,3
4,"The world’s most unique guide to alcohol-free drinks. Over 250 pages, featuring nearly 100 delicious, flavorful recipes. Brimming with beautiful, large-format photography and illustrations. Countless insights about beverage design from acclaimed Chef Grant Achatz and his brilliant culinary team. Printed and bound using cutting-edge fine art technologies. Measuring approximately 8½"" x 11½"" x 1"", weighing over 6 lbs. with a tactile hard cover. This culinary approach to beverages will be equa...",4.8,book,1733008837,0.614410,4
5,"The Complete Guide to High School English Literature brings an extraordinary and comprehensive learning experience to all high school students in the field of English language and literature through skill-by-skill instructions and case studies. Whether you are seeking a complete grammar guide or step-by-step techniques to write exemplary essays, The Complete Guide to High School English Literature meets your needs. Amirsaman Zahabioun, the author of The Complete Guide to High School English ...",5.0,book,1793074933,0.612208,5
6,"Written by leading experts in language and cognition, this groundbreaking behavior analysis textbook brings the study of verbal behavior into the 21st century with cutting-edge research. Students and clinicians in the burgeoning field of applied behavior analysis will find the theoretical foundation they need to effectively help the increasingly diverse clients seeking their services.The origins of behavior analysis can be traced to the pioneering work of B.F. Skinner. Skinner’s fundamental ...",4.8,book,B07YWGXW5X,0.610457,6
7,"Great product!, , Now featuring a new introduction by Dr. M. Scott Peck, the twenty-fifth anniversary edition of the classic bestseller The Road Less Traveled, celebrated by The Washington Post as “not just a book but a spontaneous act of generosity.” Perhaps no book in this generation has had a more profound impact on our intellectual and spiritual lives than The Road Less Traveled. With sales of more than seven million copies in the United States and Canada, and translations into more than...",4.7,book,0743243153,0.610276,7
8,"Are you missing something in your life? Do 

### Using .with_vector clauses

In [12]:
with_vector_query = (
    sl.Query(
        index,
        weights={
            description_space: sl.Param("description_weight"),
            rating_maximizer_space: sl.Param("rating_maximizer_weight"),
        },
    )
    .find(product)
    .with_vector(product, sl.Param("product_id"))
    .with_natural_query(sl.Param("natural_query"), openai_config)
    .limit(sl.Param("limit"))
)

In [13]:
with_vector_results = app.query(
    with_vector_query,
    natural_query="similar books like B07HLQQZ3L",
    limit=10,
)

In [14]:
# notice how review_id is filled with the specified id
with_vector_results.knn_params

{'description_weight': None,
 'rating_maximizer_weight': None,
 'product_id': 'B07HLQQZ3L',
 'id_looks_like_filter_weight_param_weight': 1.0,
 'limit': 10,
 'natural_query': 'similar books like B07HLQQZ3L'}

In [15]:
# the most similar review to review 130 is itself.
with_vector_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,"Cook healthy delicious dinners at home with hlp from sunvivi easy-to-use electric food steamer. Whether gathering food from a favorite store,local farmer's market, or from your very own vegetable garden,the food steamer makes it fun to indulge in nature's bounty all year long. Use the food steamer to prepare a variety of entrees and side dishes-anything from savory jasmine rice,wild-caught fish,or farm-raised chicken to the whole gamut of fresh or frozen vegetables,including artichokes,aspar...",3.6,product,B082DDK2X2,0.0,0
1,"3 Large Bags Panda Popcorn, 3 Large Bags Panda Popcorn",4.8,product,B07CXCB1T2,0.0,1
2,"Significant changes in the third edition include updated serial communication description (UART, SPI, and I2C), new serial communication examples, incorporation of GNU gcc compiler, low power modes, modification of example programs from STM32L1 (Cortex-M3) to STM32L4 (Cortex-M4). The book introduces basic programming of ARM Cortex-M cores in assembly and C at the register level, and the fundamentals of embedded system design. It presents basic concepts such as data representations (integer, ...",4.3,book,0982692668,0.0,2
3,"3-Color LED Lighting in Cloud White, Deep Blue and Amber. 304-grade Stainless Steel\nActive Cooling Technology. Bottle Capacity: 21 bottles, 64 12oz cans. Carbon Filter Air Filtration\nDoor Open Alarm. Dual-Pane, Low-e Argon-Filled Glass. Electronic Capacitive Touch Controls\nFull-Extension, Black Wood Racks with Stainless Steel Trim (6). Slide-out Gray Glass Shelves (3)\nTemperature Range: 38° - 65° F (beverage side). Temperature Range: 40° – 65° F (wine side). Vibration Dampening System, F...",4.6,product,B07GWVP5RC,0.0,3
4,MASZONE Lead The Fashion Trend:【Dresses】 maxi dress/maternity dress/plus size dresses/cocktail dresses/wedding guest dresses/beach dresses /midi dress/wrap dress/sundresses/bodycon dresses/sexy dress/boho dress/party dress/y2k dress/crop dress【Tops】crop tops/workout tops/tank tops/halter tops/off shoulder tops/cold shoulder tops/maternity tops/racer back tank tops/plus size tops/oversized tops/y2k tops【hoodies/pullover】cute frog print hoodies/graphic hoodies/anime hoodies/hoodies for teen gi...,3.3,product,B0899V2XJ9,0.0,4
5,"These empty glass hot sauce bottles are the perfect containers for your homemade heat! With a dasher top included, they're great for storing and serving hot sauce. Bottle is made of clear glass and comes with a black screw cap., 12 Empty Hot Sauce Bottles, Clear Glass, Case Packed with Partitions\nCapacity: 5 Fl Oz, Bottle Dimensions: 6.68"" Height x 1.81"" Diameter\nIncludes Dripper Inserts and Black Screw Caps with Food Grade Liner\nGreat for Bottling Your Homemade Hot Sauce!, Date First Ava...",4.8,product,B01EISFX1K,0.0,5
6,"50mm Objective Lens\n2nd Focal Plane\nBDC/Adjustable Turrets\nWaterproof and Fog proof\nZBR Reticle, Sport Type: ‎Hunting, Date First Available: July 12, 2017, Model Name: ‎Conquest V6, Part Number: ‎522241-9992-070, Item Package Dimensions L x W x H: ‎16.73 x 4.57 x 4.49 inches, Size: ‎3-18x50 / 0.25 MOA, Included Components: ‎SCOPE, Color: ‎Black, ASIN: B071JHRQQW, Material: ‎Aluminum, Best Sellers Rank: #851,566 in Sports & Outdoors ( See Top 100 in Sports & Outdoors ) #2,386 in Rifle Sco...",0.0,product,B071JHRQQW,0.0,6
7,"The Yealink W60B is the DECT IP base station for small and medium-sized businesses. Paring with up to a total of 8 Yealink W56H DECT handsets, W60B allows you enjoy superb mobility and efficient flexibility immediately as well as significantly eliminates additional wiring troubles and charges. A powerful chip ensuring a better and higher performance, this DECT IP base station not only supports up to 8 VoIP accounts and 8 concurrent calls, but also speeds up its startup and signal connection,...",5.0,product,B07F21C229,0.0,7
8,"1, 【𝗡𝗔𝗜𝗟 𝗞𝗜𝗧】3 basic acrylic powder, glitter powder, acrylic nail brush, nail glue, false nail tips, gems, and other basic nail art 

### Filling Params with default values

Now let's run a natural language query on a `Query` that contains a `Param` that has a default value. If not mentioned by the user, the `Param` will have it's user defined `default` value.

In [16]:
default_filter_results = app.query(
    query_with_default_filter,
    natural_query="high versatility, high ratings",
    limit=10,
)
default_filter_results.to_pandas()

,description,stars,type,id,similarity_score,rank
0,Article №: Keith Ti5820 ● FUNCTIONALLY SIMPLE AND ELEGANT --- Much of our design inspiration comes directly from outdoor adventurers and travelers. We seek out innovative solutions for their needs with the belief that less is more and that simplicity yields elegance. ● STRAIGHTFORWARDLY VERSATILE AND MULTI-PURPOSE --- We design our products to be used in ways expected and unexpected to save weight and space when every ounce counts. ● ENVIRONMENTALLY FRIENDLY AND HEALTHY --- We strive to mini...,4.8,product,B01NAEP534,0.646908,0
1,Infused with Argan Oil\nThermal Protector\nWeightless Formula,4.6,product,B075FBD7CM,0.644679,1
2,"Ready to wear. Arrives with a finished hemmed to avoid extra tailoring.\nTailored around your body's natural silhouette.\nTapered leg in a stretch comfort fabric for ease of movement.\nGreat for your 9-5 work week, a night out, or traveling. Wrink resistant engineered fabric.\nMultiple colors for any occasion. Pairs well with a button down and blazer.",4.4,product,B00O104CHC,0.643182,2
3,"Beautiful and contemporary design complements any décor, with large display and warm backlight for easy viewing even from across the room\nEarly start mode ensures the desired temperature is reached by the programmed time, making for a comfortable home whenever you need it\n7-Day programmability: program each day individually with up to 4 programming periods. This thermostat is not compatible with base board heaters (or other line voltage heating), multi-stage heating or cooling systems, or ...",4.4,product,B01D16HAFW,0.642925,3
4,"Breathable mesh lining\nPerformance suede leather and mesh upper\nBellows, closed-cell foam tongue keeps moisture and debris out\nMerrell M Select FIT.ECO+ blended EVA contoured footbed with added zonal arch and heel support\nVibram TC5+ sole",4.6,product,B01HFPQII0,0.641151,4
5,"70 Precise Settings for your perfect grind - from coarse for Cold Brew and French Press, to fine for Espresso. Easy reference brew method icons help to easily match the grind size to the brew method.\nAutomatic Smart Dosing Technology to seelect the cups or shots desired, grind-time is adjusted accordingly for perfect dosing, regardless of grind size or brew method.\nBuilt-In Adjustable Portafilter Holder to grind directly into 54-mm or 58-mm portafilters, perfect for espresso making.\nAnti-...",4.5,product,B08JH6K5PY,0.641132,5
6,Quick-dry knit fabric with UPF 50 for sun protection\nFull coverage\nSun-protective design\nTankini-style top\nLong-sleeve design,4.6,product,B07HL3WRG5,0.640897,6
7,"Per-key RGB backlighting and a 19-zone LightEdge across the top of the keyboard delivers dynamic and vibrant lighting effects with near-limitless customization.\nPrecision-molded 1. 5mm thick 104/105-key PBT double-shot keycap set resists wear, fading, and shine through years of gaming.\nGain an in-game advantage with six dedicated macro keys, fully programmable for complex macros and key remaps, or swap to the included S-key keycaps and program special streaming commands through Elgato Stre...",4.7,product,B082GRKQYF,0.640578,7
8,"UA Storm technology delivers an element battling, highly water resistant finish\nTough, abrasion resistant bottom panel\nFoam reinforced panels add protection\nSoft lined laptop sleeve—holds up to 15"" MacBook Pro or similarly sized laptop\nWater repellent front valuables pocket to keep your stuff safe",4.8,product,B01MF9C5VG,0.640484,8
9,"Our best foundation for long wear, Double Wear is oil-free and oil-controlling.\nThis transfer-resistant and waterproof foundation formula lasts in hot and humid weather.\nUnifies uneven skintone and covers imperfections with a natural, matte finish.\nKeeps up with your busy day. Put it on once and don’t think twice.\nSPF 10 - Dermatologist tested, non-acnegenic, won't clog pores.",4.7,product,B00B5QWFCO,0.640117,9


As we can see even though the user said nothing about the `filter_by_type` Param, nevertheless results are filtered for products as that is a `default` value of that `Param` defined previously in `query_with_default_filter`.

In [17]:
default_filter_results.knn_params

{'description_weight': 1.0,
 'rating_maximizer_weight': 1.0,
 'filter_by_type': 'product',
 'query_text': 'high versatility',
 'description_similar_clause_weight': 1.0,
 'limit': 10,
 'natural_query': 'high versatility, high ratings'}